## Tie strength prediction in online social networks

Algorithm inspired from https://dl.acm.org/doi/10.1145/1772690.1772790

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
N = 10 # number of users


In [ ]:
# x[i] = k dimensional profile vector of user i

# x is N*k dimensional tensor

# we learn 8 dimensional vector embeddings in latent vector space for these profile vectors
# using some autoencoder network
# a simple autoencoder does not take into account the relationship among users for generating embeddings

# we cannot use variational graph autoencoder since it has a GCN encoder which takes as input a graph
# but our graph is not yet ready.


In [ ]:
# we compute similarity between the vector embeddings using similarity measures like entropy or
# cosine similarity

# s[i][j] = 8 dimensional similarity vector between user i and user j

# s is N*N*8 dimensional tensor

embed_dim = 8

# s = torch.tensor([[1, 0.8, 0.2], [0.8, 1, 0.5], [0.2, 0.5, 1]], dtype=torch.float).view(N, N, embed_dim, 1)

s = torch.randn(N, N, embed_dim, 1, dtype=torch.float)

print(s)
print(s.shape)
print(torch.squeeze(s))
print(s.view(-1, embed_dim))


In [ ]:
m = 4 # number of interactions between each pair of users that we take into account


In [ ]:
# y[i][j][t] = frequency of tth interaction between user i and j, (t = 1, 2, ... m)

# y is N*N*m dimensional vector

# y = torch.tensor([[0, 3, 1], [2, 0, 2], [3, 4, 0]], dtype=torch.float).view(N, N, m, 1)

y = torch.randint(low=1, high=5, size=(N, N, m, 1)).type(torch.float)

print(y)
print(y.shape)
print(torch.squeeze(y))


In [ ]:
# z[i][j] = latent relationship strength between user i and user j

# z is a N*N dimensional tensor

z = torch.randn(N, N, 1, 1, dtype=torch.float, requires_grad=True)

print(z)
print(torch.squeeze(z))


In [ ]:
# w is an 8 dimensional weight vector that is to be learnt

w = torch.randn(N, N, embed_dim, 1, dtype=torch.float, requires_grad=True)

print(w)
print(torch.squeeze(w))
print(s.view(-1, embed_dim))
print(w.view(-1, embed_dim))
print(torch.mul(s.view(-1, embed_dim), w.view(-1, embed_dim)))
print(torch.mul(s.view(-1, embed_dim), w.view(-1, embed_dim)).shape)
print(torch.sum(torch.mul(s.view(-1, embed_dim), w.view(-1, embed_dim)), 1).view(N, N, 1, 1))
print(z)


In [ ]:
v = 0.5 # variance of the Gaussian/Normal distribution


In [ ]:
l = 5 # number of auxiliary variables

# a[i][j][p] = pth auxiliary variable for interactions between user i and user j

# a is N*N*l dimensional tensor

# a = torch.tensor([[[4], [4], [4]], [[4], [4], [4]], [[7], [7], [7]]], dtype=torch.float).view(N, N, l, 1)

a = torch.randint(low=5, high=10, size=(N, N, l, 1)).type(torch.float)

print(a)
print(a.shape)
print(a[0][0])
print(torch.squeeze(a))


In [ ]:
# b[i][j][o][q] = parameter for qth auxiliary variable for modeling oth interaction between user i and user j

# b is N*N*m*(l+1) dimensional tensor

b = torch.randn(N, N, m, l+1, dtype=torch.float, requires_grad=True)

print(b)
print(b.shape)
print(torch.squeeze(b))
print(torch.sum(b, 2))
print(b[0][0])


In [ ]:
# u is concatenation of a and z tensors

# u is N*N*(l+1) dimensional tensor

u = torch.cat((a, z), axis=2)

# print(u.shape)
# print(u)
# print(torch.squeeze(u))


In [ ]:
# lambda[i][j][t] is poisson parameter for tth interaction between user i and user j

# lambda is N*N*m dimensional vector

# lambda = exp(prelambda)

prelambda = torch.randn(N, N, m, 1, dtype=torch.float)


In [ ]:
gammaw = 1
gammab = 100


In [ ]:
# Loss function

def calculateLoss(w, z, b):

  u = torch.cat((a, z), axis=2)

  prelambda = torch.randn(N, N, m, 1, dtype=torch.float)

  for i in range(N):
    for j in range(N):
      prelambda[i][j] = torch.mm(b[i][j], u[i][j])

  # print(prelambda)

  t1 = torch.sum(-(torch.sum(torch.mul(s.view(-1, embed_dim), w.view(-1, embed_dim)), 1).view(N, N, 1, 1) - (z**2))/(2*v))

  # print(t1)

  t211 = torch.mul(y, prelambda) # N*N*m*1

  # print(t211)

  t212 = torch.exp(prelambda) # N*N*m*1

  # print(t212)

  t213 = torch.log(torch.exp(torch.lgamma(torch.add(y, 1)))) # N*N*m*1

  # print(t213)

  t21 = torch.sum(t211 - t212 - t213, 2) # N*N

  t22 = (gammaw/2)*(torch.sum(torch.mul(w.view(-1, embed_dim), w.view(-1, embed_dim)).view(N, N, embed_dim), 2)) # N*N

  t23 = torch.sum(torch.mul(torch.sum(torch.mul(b.view(-1, l+1), b.view(-1, l+1)).view(N, N, m, l+1), 3), gammab/2), 2) # N*N

  t2 = torch.sum(t21 - t22 - t23)

  # print(t2)

  L = t1 + t2

  print(L)

  z_grad_2 = torch.add(-(1/v), -torch.sum(torch.mul(torch.squeeze(torch.exp(prelambda)), (b[:, :, :, l]**2)), 2)) # N*N

  b_grad_2 = torch.randn(N, N, m, l+1, dtype=torch.float)

  bt1 = torch.exp(prelambda)
  bt2 = torch.mul(u.view(-1, l+1), u.view(-1, l+1)).view(N, N, l+1, 1)

  for i in range(N):
    for j in range(N):
      b_grad_2[i][j] = torch.mm(bt1[i][j], bt2[i][j].t()) - gammab

  # b_grad_2 = torch.mm(torch.exp(prelambda).view(-1, m), torch.mul(u.view(-1, l+1), u.view(-1, l+1)).t()) - gammab # 1*1

  return (L, z_grad_2, b_grad_2)


In [ ]:
num_iterations = 100 # number of iterations of coordinate ascent optimization algorithm


In [ ]:
alpha = 0.00001 # learning rate


In [ ]:
L_list = []

for i in range(num_iterations):
  L, z_grad_2, b_grad_2 = calculateLoss(w, z, b)
  L.backward()
  L_list += [L.item()]
  with torch.no_grad():
    z += alpha*((torch.squeeze(z.grad)/z_grad_2).view(N, N, 1, 1))
    b += alpha*(b.grad/b_grad_2)
    w += alpha*w.grad


In [ ]:
plt.plot(np.array(L_list))


In [ ]:
z = torch.squeeze(z)
print(z)
